In [1]:
#load libraries and the classification model

import tensorflow as tf
import os
import cv2
import imghdr
import numpy as np
from matplotlib import pyplot as plt
import shutil
from PIL import Image
from tensorflow.keras.models import load_model
model = load_model('avo_model.keras')
from ultralytics import YOLO


C:\Users\johnl\AppData\Local\Temp\ipykernel_9388\1336455791.py:6: DeprecationWarning: 'imghdr' is deprecated and slated for removal in Python 3.13
  import imghdr


First we run the yolo model to identify the kind of avo. Since we can only predict Hass, maybe I need something to account for this. 

Next we move these images into a separate checking folder. This is to make the rest of the code sustainable and work across all devices consistently. This will move all the Hass crops into a separate checking folder (or shepards at a later date, may need an if else or something) and then delete the runs folder to avoid having the future cops go into 'predict2' and then having to keep track of the number of predicts.

In [4]:
model_yolo = YOLO("avoID.pt")

model_yolo.predict(
        source='WhatsApp Image 2025-05-06 at 8.29.14 PM.jpeg',
        save_crop=True,
        save=True,
        conf=0.5
    )
#!yolo predict model=avoID.pt source='Pictures/1020' conf = 0.5 save_crop=True save
shutil.move('runs/detect/predict/crops/Hass','checking_folder/') #need to add a line if there are no hass' detected
shutil.rmtree('runs/')


image 1/1 c:\Users\johnl\Documents\GitHub\avoChecker\WhatsApp Image 2025-05-06 at 8.29.14 PM.jpeg: 640x480 1 Hass, 60.5ms
Speed: 6.9ms preprocess, 60.5ms inference, 2.3ms postprocess per image at shape (1, 3, 640, 480)
Results saved to runs\detect\predict


The next steps: 
1) Define a function to resize the images inputed
2) open checking_folder and store the names of each file in a list
3) loop through that list and resize each image
4) run the image through the algo and save the score
5) get an average of the final score as the score for the avos

In [3]:
#This is to resize the image prior to running it through the model
def prepare_input(image_path, target_size=(256, 256)): #adjust target size to match model input.
    img = Image.open(image_path).resize(target_size)
    img_array = np.array(img) / 255.0  # Normalize pixel values
    img_array = np.expand_dims(img_array, axis=0) # Add batch dimension
    return img_array

avo_pics = os.listdir('checking_folder/')

avo_score = [] #empty list to store the avo score

#Loop through all the pics in the folder to get the individual predictions for each avo
for pic in avo_pics:
    avo = prepare_input('checking_folder/' + pic)
    avo_score.append(model.predict(avo))

shutil.rmtree('checking_folder/') #delete the checking_folder to ensure we can use exactly the same code in the futur

avo_score = np.concatenate(avo_score, axis=0 )
avo_score.mean()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 296ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step


3.6865675

In [3]:
from pathlib import Path
file_name = "WhatsApp Image 2025-05-06 at 8.29.14 PM.jpeg"
name = Path(file_name).stem
print(name) 

WhatsApp Image 2025-05-06 at 8.29.14 PM
